In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import Chroma, FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = "\n",
    chunk_size = 300,
    chunk_overlap = 50,
)

loader = UnstructuredFileLoader('./files/george_orwell.csv')

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs,cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer question using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
    ("human","{question}")
])

chain = {
    "context":retriever,
    "question":RunnablePassthrough(),
} | prompt | llm


chain.invoke("Describe Victory Mansions?")

AIMessage(content="Victory Mansions is a building with glass doors that let in gritty dust. The hallway smells of boiled cabbage and old rag mats. There is a large colored poster of a man's face on the wall. The building has seven flights of stairs, and the elevator is usually not working due to a cut in the electric current. The poster with the enormous face is present on each landing, and it seems to follow you with its eyes as you move.")